# **Perspectiva General de Mercado - DatAlpine**
*Filtro de tipo de mercado*

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
import numpy as np

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()
# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "alfa" in nombre and "tulancingo" in nombre and '2024' in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_tulancingo',
 'df_alfa_agosto_2024_tulancingo',
 'df_alfa_febrero_2024_tulancingo',
 'df_alfa_julio_2024_tulancingo',
 'df_alfa_junio_2024_tulancingo',
 'df_alfa_marzo_2024_tulancingo',
 'df_alfa_mayo_2024_tulancingo']

In [2]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    df.rename(columns={'Categoria':'categoria','Category':'categoria','Propiedad':'propiedad','Precio':'precio','m_total':'metros_total','m_construido':'metros_construido','Estacionamientos':'estacionamiento','Recamaras':'recamaras','Banos':'baño','Medio_banos':'medio_baño','Banos_Total':'baño_total'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 
print("Columnas renombradas en los DataFrames filtrados.")

Columnas renombradas en los DataFrames filtrados.


In [3]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id','Tipo','categoria','propiedad','precio','metros_total','metros_construido', 'precio_m2_construido','estacionamiento','recamaras', 'baño', 'medio_baño', 'baño_total']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

In [4]:
df_total = pd.concat(dataframes_list, ignore_index=True)

In [5]:
df_total.columns

Index(['id', 'Tipo', 'categoria', 'propiedad', 'precio', 'metros_total',
       'metros_construido', 'precio_m2_construido', 'estacionamiento',
       'recamaras', 'baño', 'medio_baño', 'baño_total'],
      dtype='object')

In [6]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 937 entries, 0 to 936
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    937 non-null    int64  
 1   Tipo                  937 non-null    object 
 2   categoria             937 non-null    object 
 3   propiedad             937 non-null    object 
 4   precio                937 non-null    int64  
 5   metros_total          937 non-null    int64  
 6   metros_construido     937 non-null    int64  
 7   precio_m2_construido  937 non-null    float64
 8   estacionamiento       937 non-null    int64  
 9   recamaras             937 non-null    int64  
 10  baño                  937 non-null    int64  
 11  medio_baño            937 non-null    int64  
 12  baño_total            937 non-null    float64
dtypes: float64(2), int64(8), object(3)
memory usage: 95.3+ KB


In [7]:
# Definición de rangos de precio
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500001, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),
    "B2": (2750001, 3000000),
    "B3": (3000001, 3250000),
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf')),
}

# Función para asignar la categoría según el precio
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio <= limite_superior:  # Cambiado a <=
            return categoria
    return None  # Retornar None para manejarlo después

# Limpieza del DataFrame
df_total = df_total.drop(columns=['categoria'], errors='ignore')

# Convertir 'precio' a numérico y manejar errores
df_total['precio'] = pd.to_numeric(df_total['precio'], errors='coerce')
df_total = df_total.dropna(subset=['precio'])  # Eliminar filas con precios NaN

# Asignar categorías
df_total['categoria'] = df_total['precio'].apply(asignar_categoria)

# Verificar categorías inválidas
categorias_invalidas = df_total[df_total['categoria'].isnull()]
if not categorias_invalidas.empty:
    print("Precios sin categoría:")
    print(categorias_invalidas[['precio']])

In [ ]:
df_total.loc[:, 'segmento'] = df_total['categoria'].apply(lambda x: 'ELITE' if x == 'ELITE' else x[0])

In [8]:
import numpy as np 
condition = (df_total['baño_total'] % 1 == 0.5) # Valores que contienen .5    
# Convertir al número entero anterior
df_total.loc[condition, 'baño_total'] = np.floor(df_total['baño_total']) 

# Limitar los valores  
df_total['baño_total'] = df_total['baño_total'].clip(lower=1) #,upper=10
df_total['estacionamiento'] = df_total['estacionamiento'].clip(lower=1) #upper=8
df_total['recamaras'] = df_total['recamaras'].clip(lower=1)#upper=8 

df_total = df_total[df_total['precio'] != 0]
df_total = df_total[df_total['metros_total'] != 0]
df_total = df_total[df_total['metros_construido'] != 0]

df_total = df_total[~df_total['categoria'].str.contains('L')]

- Casa promedio
- Casa Mediana
- Precio mercado

- m2 Terreno
- M2 Construcción
- Precio m2 construcción

In [9]:
df_total['Tipo'].unique()

array(['Casa', 'Casa En Fraccionamiento', 'Departamento',
       'Casa En Condominio', 'Condominio Horizontal'], dtype=object)

In [10]:
casa_df = df_total[df_total['Tipo'].isin(['Casa', 'Casa En Fraccionamiento', 'Casa En Condominio', 'Casa Duplex'])]
depa_df = df_total[df_total['Tipo'].isin(['Departamento','Duplex','Condominio Horizontal'])]

# Casas

### Revisión de m2

In [11]:
# metros_total en casas
precios = casa_df['metros_total'].unique()
precios = np.sort(precios)
precios

array([  66,   70,   72,   75,   80,   81,   83,   90,   91,   92,   95,
         96,  100,  105,  108,  110,  114,  115,  117,  119,  120,  122,
        125,  126,  128,  129,  130,  133,  140,  142,  143,  150,  156,
        160,  165,  168,  170,  171,  175,  178,  180,  189,  190,  200,
        202,  204,  216,  220,  225,  228,  230,  232,  240,  245,  250,
        266,  280,  285,  300,  330,  350,  360,  375,  398,  400,  420,
        482, 1049, 2500])

In [12]:
# metros_construido en casas
precios = casa_df['metros_construido'].unique()
precios = np.sort(precios)
precios

array([ 45,  46,  50,  52,  53,  55,  58,  60,  61,  62,  64,  66,  67,
        68,  69,  70,  71,  73,  75,  76,  77,  78,  80,  81,  83,  90,
        95, 100, 102, 105, 110, 114, 120, 125, 128, 130, 135, 137, 140,
       144, 150, 154, 159, 160, 168, 170, 176, 178, 180, 182, 185, 190,
       199, 200, 204, 205, 206, 210, 216, 218, 220, 225, 228, 230, 240,
       241, 250, 260, 266, 280, 287, 300, 320, 325, 330, 350, 360, 370,
       375, 398, 400, 404, 420, 430, 460, 600])

In [13]:
# metros_total en casas cuando segmento es 'B'
#precios = casa_df[casa_df['segmento'] == 'B']['metros_total'].unique()
precios = casa_df[casa_df['segmento'] == 'B']['metros_total'].value_counts()
#precios = np.sort(precios)
precios

KeyError: 'segmento'

## Casa promedio

In [14]:
casa_df.loc[:, 'segmento'] = casa_df['categoria'].str[0]
grupo_promedios = round(casa_df.groupby('segmento')[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
grupo_promedios.astype(int)
# Renombrar las columnas para mayor claridad
grupo_promedios.rename(columns={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)

# Calcular el promedio de estacionamientos, recámaras y baños en general (sin segmento específico)
promedios_generales = round(casa_df[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
promedios_generales = promedios_generales.astype(int)
# Renombrar las columnas para mayor claridad
promedios_generales.rename(index={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
# Agregar los promedios generales al dataframe de grupo_promedios
promedios_generales = pd.DataFrame(promedios_generales).T
promedios_generales['segmento'] = 'general'

casa_promedio = pd.concat([grupo_promedios, promedios_generales.set_index('segmento')], ignore_index=False)

#print(grupo_promedios)
#print(promedios_generales)
print(casa_promedio)

# Convertir el dataframe a JSON
grupo_promedios_json = casa_promedio.to_json(orient='index')

# Guardar el JSON en un archivo
with open('assets/tablas/grupo_casa_promedio.json', 'w') as f:
    f.write(grupo_promedios_json)

          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        4.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    2.0
E                      2.0        3.0    2.0
S                      2.0        4.0    3.0
general                2.0        3.0    2.0


C:\Users\yoe11\AppData\Local\Temp\ipykernel_7828\1301130619.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  casa_df.loc[:, 'segmento'] = casa_df['categoria'].str[0]


## Precio Mercado

In [15]:
casa_df.loc[:, 'segmento'] = casa_df['categoria'].str[0].astype(str)
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S']#, 'L'
grupo_descripcion_casa = casa_df.groupby('segmento')['precio'].describe().reset_index()
grupo_descripcion_casa = grupo_descripcion_casa.loc[:, ['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion_casa.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
# Ordenar el dataframe según el orden específico de segmentos
ordenado = grupo_descripcion_casa.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,371800.0,1000000.0,6.554863e+05
1,D,1020000.0,1750000.0,1.275413e+06
2,C,1800000.0,2500000.0,2.123983e+06
3,B,2600000.0,3200000.0,2.900515e+06
4,A,3300000.0,3946465.0,3.606464e+06
5,S,4180000.0,10000000.0,5.130098e+06


In [16]:
# Función para formatear valores en formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
grupo_descripcion_casa 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_7828\3321038986.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    $3,300,000.00
1    $2,600,000.00
2    $1,800,000.00
3    $1,020,000.00
4      $371,800.00
5    $4,180,000.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_7828\3321038986.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $3,946,465.00
1     $3,200,000.00
2     $2,500,000.00
3     $1,750,000.00
4     $1,000,000.00
5    $10,000,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descrip

,segmento,Mínimo,Máximo,Promedio
0,A,"$3,300,000.00","$3,946,465.00","$3,606,463.67"
1,B,"$2,600,000.00","$3,200,000.00","$2,900,515.15"
2,C,"$1,800,000.00","$2,500,000.00","$2,123,982.84"
3,D,"$1,020,000.00","$1,750,000.00","$1,275,412.56"
4,E,"$371,800.00","$1,000,000.00","$655,486.32"
5,S,"$4,180,000.00","$10,000,000.00","$5,130,097.56"


In [17]:
# Paso 1: Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()

# Paso 2: Seleccionar las columnas deseadas
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']]

# Paso 3: Renombrar las columnas
#grupo_descripcion.rename(columns={
#    'min': 'Mínimo',
#    'max': 'Máximo',
#    'mean': 'Promedio',
#}, inplace=True)

# Paso 4: Calcular el percentil 95% para cada segmento
#percentiles_95 = casa_df.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Mostrar el resultado final
#print(grupo_descripcion)

In [18]:
# Filtrar los datos según las condiciones especificadas
#filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Calcular las estadísticas descriptivas
todo_descripcion = casa_df['precio'].describe()[['min', 'max', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general' # Agregar la columna de segmento
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Calcular el percentil 95%
#percentil_95 = filtrado_df['precio_m2_construido'].quantile(0.95)
# Reemplazar el valor máximo con el percentil 95%
#todo_descripcion['Máximo'] = percentil_95

# Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
print(todo_descripcion)

             Mínimo          Máximo       Promedio segmento
precio  $371,800.00  $10,000,000.00  $1,418,996.98  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_7828\2555968983.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio    $371,800.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_7828\2555968983.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio    $10,000,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_7828\2555968983.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise 

In [19]:
# Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']] # Seleccionar las columnas deseadas
#grupo_descripcion.rename(columns={'min': 'Mínimo','max': 'Máximo', 'mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% para cada segmento
#percentiles_95 = df_total.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Paso 7: Calcular las estadísticas generales
#general_stats = df_total['precio'].describe()[['min', 'max', 'mean']].to_frame().T
#general_stats.rename(columns={'min': 'Mínimo','max': 'Máximo','mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% general
#general_percentil_95 = df_total['precio'].quantile(0.95)

# Reemplazar el máximo general con el percentil 95%
#general_stats['Máximo'] = general_percentil_95

# Agregar una fila para los resultados generales
#general_stats['segmento'] = 'General'

# Concatenar los resultados generales al DataFrame de segmentos
#resultado_final = pd.concat([grupo_descripcion, general_stats], ignore_index=True)
#print(resultado_final)

In [20]:
resultado_final = pd.concat([grupo_descripcion_casa, todo_descripcion], ignore_index=True)
print(resultado_final)

  segmento         Mínimo          Máximo       Promedio
0        A  $3,300,000.00   $3,946,465.00  $3,606,463.67
1        B  $2,600,000.00   $3,200,000.00  $2,900,515.15
2        C  $1,800,000.00   $2,500,000.00  $2,123,982.84
3        D  $1,020,000.00   $1,750,000.00  $1,275,412.56
4        E    $371,800.00   $1,000,000.00    $655,486.32
5        S  $4,180,000.00  $10,000,000.00  $5,130,097.56
6  general    $371,800.00  $10,000,000.00  $1,418,996.98


In [21]:
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S','general']#, 'L'
ordenado = resultado_final.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,"$371,800.00","$1,000,000.00","$655,486.32"
1,D,"$1,020,000.00","$1,750,000.00","$1,275,412.56"
2,C,"$1,800,000.00","$2,500,000.00","$2,123,982.84"
3,B,"$2,600,000.00","$3,200,000.00","$2,900,515.15"
4,A,"$3,300,000.00","$3,946,465.00","$3,606,463.67"
5,S,"$4,180,000.00","$10,000,000.00","$5,130,097.56"
6,general,"$371,800.00","$10,000,000.00","$1,418,996.98"


In [22]:
# Guardar el JSON en un archivo
resultados_finales_desc_json = resultado_final.to_json(orient='index')
with open('assets/tablas/grupo_precio_mercado_casa.json', 'w') as f:
    f.write(resultados_finales_desc_json)

## M2 Terreno

In [23]:
casa_df.loc[:, 'segmento'] = casa_df['categoria'].str[0].astype(str)
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S']#, 'L'
grupo_descripcion_casa = casa_df.groupby('segmento')['metros_total'].describe().reset_index()
grupo_descripcion_casa = grupo_descripcion_casa.loc[:, ['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion_casa.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
# Ordenar el dataframe según el orden específico de segmentos
ordenado = grupo_descripcion_casa.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,66.0,398.0,143.970530
1,D,70.0,400.0,138.122302
2,C,80.0,400.0,179.315068
3,B,70.0,232.0,171.333333
4,A,150.0,1049.0,277.591837
5,S,160.0,2500.0,547.439024


In [24]:
# Función para formatear valores en formato monetario por m2
def formato_monetario_m2(valor):
    return f"{valor:,.2f} m²"  # Formato con $, 2 decimales, separador de miles y unidad m²

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
grupo_descripcion_casa 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_7828\3382106320.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    150.00 m²
1     70.00 m²
2     80.00 m²
3     70.00 m²
4     66.00 m²
5    160.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_7828\3382106320.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    1,049.00 m²
1      232.00 m²
2      400.00 m²
3      400.00 m²
4      398.00 m²
5    2,500.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).appl

,segmento,Mínimo,Máximo,Promedio
0,A,150.00 m²,"1,049.00 m²",277.59 m²
1,B,70.00 m²,232.00 m²,171.33 m²
2,C,80.00 m²,400.00 m²,179.32 m²
3,D,70.00 m²,400.00 m²,138.12 m²
4,E,66.00 m²,398.00 m²,143.97 m²
5,S,160.00 m²,"2,500.00 m²",547.44 m²


In [25]:
# Paso 1: Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()

# Paso 2: Seleccionar las columnas deseadas
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']]

# Paso 3: Renombrar las columnas
#grupo_descripcion.rename(columns={
#    'min': 'Mínimo',
#    'max': 'Máximo',
#    'mean': 'Promedio',
#}, inplace=True)

# Paso 4: Calcular el percentil 95% para cada segmento
#percentiles_95 = casa_df.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Mostrar el resultado final
#print(grupo_descripcion)

In [26]:
# Filtrar los datos según las condiciones especificadas
#filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Calcular las estadísticas descriptivas
todo_descripcion = casa_df['metros_total'].describe()[['min', 'max', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general' # Agregar la columna de segmento
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Calcular el percentil 95%
#percentil_95 = filtrado_df['precio_m2_construido'].quantile(0.95)
# Reemplazar el valor máximo con el percentil 95%
#todo_descripcion['Máximo'] = percentil_95


# Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"{valor:,.2f} m²"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
print(todo_descripcion)

                Mínimo       Máximo   Promedio segmento
metros_total  66.00 m²  2,500.00 m²  174.37 m²  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_7828\1147424691.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    66.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_7828\1147424691.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    2,500.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_7828\1147424691.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will 

In [27]:
# Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']] # Seleccionar las columnas deseadas
#grupo_descripcion.rename(columns={'min': 'Mínimo','max': 'Máximo', 'mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% para cada segmento
#percentiles_95 = df_total.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Paso 7: Calcular las estadísticas generales
#general_stats = df_total['precio'].describe()[['min', 'max', 'mean']].to_frame().T
#general_stats.rename(columns={'min': 'Mínimo','max': 'Máximo','mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% general
#general_percentil_95 = df_total['precio'].quantile(0.95)

# Reemplazar el máximo general con el percentil 95%
#general_stats['Máximo'] = general_percentil_95

# Agregar una fila para los resultados generales
#general_stats['segmento'] = 'General'

# Concatenar los resultados generales al DataFrame de segmentos
#resultado_final = pd.concat([grupo_descripcion, general_stats], ignore_index=True)
#print(resultado_final)

In [28]:
resultado_final = pd.concat([grupo_descripcion_casa, todo_descripcion], ignore_index=True)
print(resultado_final)

  segmento     Mínimo       Máximo   Promedio
0        A  150.00 m²  1,049.00 m²  277.59 m²
1        B   70.00 m²    232.00 m²  171.33 m²
2        C   80.00 m²    400.00 m²  179.32 m²
3        D   70.00 m²    400.00 m²  138.12 m²
4        E   66.00 m²    398.00 m²  143.97 m²
5        S  160.00 m²  2,500.00 m²  547.44 m²
6  general   66.00 m²  2,500.00 m²  174.37 m²


In [29]:
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S','general']#, 'L'
ordenado = resultado_final.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,66.00 m²,398.00 m²,143.97 m²
1,D,70.00 m²,400.00 m²,138.12 m²
2,C,80.00 m²,400.00 m²,179.32 m²
3,B,70.00 m²,232.00 m²,171.33 m²
4,A,150.00 m²,"1,049.00 m²",277.59 m²
5,S,160.00 m²,"2,500.00 m²",547.44 m²
6,general,66.00 m²,"2,500.00 m²",174.37 m²


In [30]:
# Guardar el JSON en un archivo
resultados_finales_desc_json = resultado_final.to_json(orient='index')
with open('assets/tablas/grupo_terreno_casa.json', 'w') as f:
    f.write(resultados_finales_desc_json)

## M2 Construcción

In [31]:
casa_df.loc[:, 'segmento'] = casa_df['categoria'].str[0].astype(str)
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S']#, 'L'
grupo_descripcion_casa = casa_df.groupby('segmento')['metros_construido'].describe().reset_index()
grupo_descripcion_casa = grupo_descripcion_casa.loc[:, ['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion_casa.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
# Ordenar el dataframe según el orden específico de segmentos
ordenado = grupo_descripcion_casa.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,45.0,420.0,180.836935
1,D,62.0,420.0,141.532374
2,C,90.0,404.0,207.369863
3,B,135.0,600.0,205.287879
4,A,150.0,400.0,251.714286
5,S,228.0,460.0,340.902439


In [32]:
# Función para formatear valores en formato monetario por m2
def formato_monetario_m2(valor):
    return f"{valor:,.2f} m²"  # Formato con $, 2 decimales, separador de miles y unidad m²

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
grupo_descripcion_casa 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_7828\3382106320.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    150.00 m²
1    135.00 m²
2     90.00 m²
3     62.00 m²
4     45.00 m²
5    228.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_7828\3382106320.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    400.00 m²
1    600.00 m²
2    404.00 m²
3    420.00 m²
4    420.00 m²
5    460.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_mo

,segmento,Mínimo,Máximo,Promedio
0,A,150.00 m²,400.00 m²,251.71 m²
1,B,135.00 m²,600.00 m²,205.29 m²
2,C,90.00 m²,404.00 m²,207.37 m²
3,D,62.00 m²,420.00 m²,141.53 m²
4,E,45.00 m²,420.00 m²,180.84 m²
5,S,228.00 m²,460.00 m²,340.90 m²


In [33]:
# Paso 1: Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()

# Paso 2: Seleccionar las columnas deseadas
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']]

# Paso 3: Renombrar las columnas
#grupo_descripcion.rename(columns={
#    'min': 'Mínimo',
#    'max': 'Máximo',
#    'mean': 'Promedio',
#}, inplace=True)

# Paso 4: Calcular el percentil 95% para cada segmento
#percentiles_95 = casa_df.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Mostrar el resultado final
#print(grupo_descripcion)

In [34]:
# Filtrar los datos según las condiciones especificadas
#filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Calcular las estadísticas descriptivas
todo_descripcion = casa_df['metros_construido'].describe()[['min', 'max', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general' # Agregar la columna de segmento
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Calcular el percentil 95%
#percentil_95 = filtrado_df['precio_m2_construido'].quantile(0.95)
# Reemplazar el valor máximo con el percentil 95%
#todo_descripcion['Máximo'] = percentil_95


# Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"{valor:,.2f} m²"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
print(todo_descripcion)

                     Mínimo     Máximo   Promedio segmento
metros_construido  45.00 m²  600.00 m²  190.10 m²  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_7828\2239152448.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_construido    45.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_7828\2239152448.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_construido    600.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_7828\2239152448.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated a

In [35]:
# Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']] # Seleccionar las columnas deseadas
#grupo_descripcion.rename(columns={'min': 'Mínimo','max': 'Máximo', 'mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% para cada segmento
#percentiles_95 = df_total.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Paso 7: Calcular las estadísticas generales
#general_stats = df_total['precio'].describe()[['min', 'max', 'mean']].to_frame().T
#general_stats.rename(columns={'min': 'Mínimo','max': 'Máximo','mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% general
#general_percentil_95 = df_total['precio'].quantile(0.95)

# Reemplazar el máximo general con el percentil 95%
#general_stats['Máximo'] = general_percentil_95

# Agregar una fila para los resultados generales
#general_stats['segmento'] = 'General'

# Concatenar los resultados generales al DataFrame de segmentos
#resultado_final = pd.concat([grupo_descripcion, general_stats], ignore_index=True)
#print(resultado_final)

In [36]:
resultado_final = pd.concat([grupo_descripcion_casa, todo_descripcion], ignore_index=True)
print(resultado_final)

  segmento     Mínimo     Máximo   Promedio
0        A  150.00 m²  400.00 m²  251.71 m²
1        B  135.00 m²  600.00 m²  205.29 m²
2        C   90.00 m²  404.00 m²  207.37 m²
3        D   62.00 m²  420.00 m²  141.53 m²
4        E   45.00 m²  420.00 m²  180.84 m²
5        S  228.00 m²  460.00 m²  340.90 m²
6  general   45.00 m²  600.00 m²  190.10 m²


In [37]:
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S','general']#, 'L'
ordenado = resultado_final.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,45.00 m²,420.00 m²,180.84 m²
1,D,62.00 m²,420.00 m²,141.53 m²
2,C,90.00 m²,404.00 m²,207.37 m²
3,B,135.00 m²,600.00 m²,205.29 m²
4,A,150.00 m²,400.00 m²,251.71 m²
5,S,228.00 m²,460.00 m²,340.90 m²
6,general,45.00 m²,600.00 m²,190.10 m²


In [38]:
# Guardar el JSON en un archivo
resultados_finales_desc_json = resultado_final.to_json(orient='index')
with open('assets/tablas/grupo_construidos_casa.json', 'w') as f:
    f.write(resultados_finales_desc_json)

## Pecio M2 Construcción

In [39]:
casa_df['precio_m2_construido']

0      17391.30
1      13361.11
2      15679.44
3      17916.67
4      26309.77
         ...   
932    13154.88
933    26309.77
934    18333.33
935    17916.67
936    15679.44
Name: precio_m2_construido, Length: 877, dtype: float64

In [40]:
casa_df.loc[:, 'segmento'] = casa_df['categoria'].str[0].astype(str)
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S']#, 'L'
grupo_descripcion_casa = casa_df.groupby('segmento')['precio_m2_construido'].describe().reset_index()
grupo_descripcion_casa = grupo_descripcion_casa.loc[:, ['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion_casa.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
# Ordenar el dataframe según el orden específico de segmentos
ordenado = grupo_descripcion_casa.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,1023.81,15806.45,5191.964303
1,D,2607.14,18548.39,11050.453885
2,C,4918.81,21000.00,11566.855479
3,B,5333.33,22857.14,16923.808485
4,A,8912.50,26309.77,15548.584490
5,S,11333.33,23809.52,15169.981707


In [41]:
# Función para formatear valores en formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
grupo_descripcion_casa 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_7828\3321038986.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $8,912.50
1     $5,333.33
2     $4,918.81
3     $2,607.14
4     $1,023.81
5    $11,333.33
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_7828\3321038986.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    $26,309.77
1    $22,857.14
2    $21,000.00
3    $18,548.39
4    $15,806.45
5    $23,809.52
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(f

,segmento,Mínimo,Máximo,Promedio
0,A,"$8,912.50","$26,309.77","$15,548.58"
1,B,"$5,333.33","$22,857.14","$16,923.81"
2,C,"$4,918.81","$21,000.00","$11,566.86"
3,D,"$2,607.14","$18,548.39","$11,050.45"
4,E,"$1,023.81","$15,806.45","$5,191.96"
5,S,"$11,333.33","$23,809.52","$15,169.98"


In [42]:
# Paso 1: Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()

# Paso 2: Seleccionar las columnas deseadas
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']]

# Paso 3: Renombrar las columnas
#grupo_descripcion.rename(columns={
#    'min': 'Mínimo',
#    'max': 'Máximo',
#    'mean': 'Promedio',
#}, inplace=True)

# Paso 4: Calcular el percentil 95% para cada segmento
#percentiles_95 = casa_df.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Mostrar el resultado final
#print(grupo_descripcion)

In [43]:
# Filtrar los datos según las condiciones especificadas
#filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Calcular las estadísticas descriptivas
todo_descripcion = casa_df['precio_m2_construido'].describe()[['min', 'max', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general' # Agregar la columna de segmento
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Calcular el percentil 95%
#percentil_95 = filtrado_df['precio_m2_construido'].quantile(0.95)
# Reemplazar el valor máximo con el percentil 95%
#todo_descripcion['Máximo'] = percentil_95

# Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
print(todo_descripcion)

                         Mínimo      Máximo   Promedio segmento
precio_m2_construido  $1,023.81  $26,309.77  $8,579.16  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_7828\155160619.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $1,023.81
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_7828\155160619.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $26,309.77
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_7828\155160619.py:23: FutureWarning: Setting an item of incompatible dtype is depreca

In [44]:
# Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']] # Seleccionar las columnas deseadas
#grupo_descripcion.rename(columns={'min': 'Mínimo','max': 'Máximo', 'mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% para cada segmento
#percentiles_95 = df_total.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Paso 7: Calcular las estadísticas generales
#general_stats = df_total['precio'].describe()[['min', 'max', 'mean']].to_frame().T
#general_stats.rename(columns={'min': 'Mínimo','max': 'Máximo','mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% general
#general_percentil_95 = df_total['precio'].quantile(0.95)

# Reemplazar el máximo general con el percentil 95%
#general_stats['Máximo'] = general_percentil_95

# Agregar una fila para los resultados generales
#general_stats['segmento'] = 'General'

# Concatenar los resultados generales al DataFrame de segmentos
#resultado_final = pd.concat([grupo_descripcion, general_stats], ignore_index=True)
#print(resultado_final)

In [45]:
resultado_final = pd.concat([grupo_descripcion_casa, todo_descripcion], ignore_index=True)
print(resultado_final)

  segmento      Mínimo      Máximo    Promedio
0        A   $8,912.50  $26,309.77  $15,548.58
1        B   $5,333.33  $22,857.14  $16,923.81
2        C   $4,918.81  $21,000.00  $11,566.86
3        D   $2,607.14  $18,548.39  $11,050.45
4        E   $1,023.81  $15,806.45   $5,191.96
5        S  $11,333.33  $23,809.52  $15,169.98
6  general   $1,023.81  $26,309.77   $8,579.16


In [46]:
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S','general']#, 'L'
ordenado = resultado_final.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,"$1,023.81","$15,806.45","$5,191.96"
1,D,"$2,607.14","$18,548.39","$11,050.45"
2,C,"$4,918.81","$21,000.00","$11,566.86"
3,B,"$5,333.33","$22,857.14","$16,923.81"
4,A,"$8,912.50","$26,309.77","$15,548.58"
5,S,"$11,333.33","$23,809.52","$15,169.98"
6,general,"$1,023.81","$26,309.77","$8,579.16"


In [47]:
# Guardar el JSON en un archivo
resultados_finales_desc_json = resultado_final.to_json(orient='index')
with open('assets/tablas/grupo_precio_construidos_casa.json', 'w') as f:
    f.write(resultados_finales_desc_json)

# Depas

### Revisión de m2

In [48]:
# metros_total en casas
precios = depa_df['metros_total'].unique()
precios = np.sort(precios)
precios

array([ 34,  41,  44,  46,  48,  49,  53,  55,  60,  80,  82,  85,  87,
        90,  95,  96, 110, 120, 156])

In [49]:
# metros_construido en casas
precios = depa_df['metros_construido'].unique()
precios = np.sort(precios)
precios

array([ 34,  41,  44,  46,  48,  49,  53,  55,  60,  75,  80,  82,  85,
        87,  90,  95, 110, 120, 156])

In [50]:
# metros_total en casas cuando segmento es 'B'
#precios = casa_df[casa_df['segmento'] == 'B']['metros_total'].unique()
precios = depa_df[depa_df['segmento'] == 'B']['metros_total'].value_counts()
#precios = np.sort(precios)
precios

KeyError: 'segmento'

## Depa promedio

In [70]:
depa_df.loc[:, 'segmento'] = depa_df['categoria'].str[0]
grupo_promedios = round(depa_df.groupby('segmento')[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
grupo_promedios.astype(int)
# Renombrar las columnas para mayor claridad
grupo_promedios.rename(columns={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)

# Calcular el promedio de estacionamientos, recámaras y baños en general (sin segmento específico)
promedios_generales = round(depa_df[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
promedios_generales = promedios_generales.astype(int)
# Renombrar las columnas para mayor claridad
promedios_generales.rename(index={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
# Agregar los promedios generales al dataframe de grupo_promedios
promedios_generales = pd.DataFrame(promedios_generales).T
promedios_generales['segmento'] = 'general'

casa_promedio = pd.concat([grupo_promedios, promedios_generales.set_index('segmento')], ignore_index=False)

#print(grupo_promedios)
#print(promedios_generales)
print(casa_promedio)

# Convertir el dataframe a JSON
grupo_promedios_json = casa_promedio.to_json(orient='index')

# Guardar el JSON en un archivo
with open('assets/tablas/grupo_depa_promedio.json', 'w') as f:
    f.write(grupo_promedios_json)

          Estacionamientos  Recámaras  Baños
segmento                                    
B                      2.0        2.0    2.0
C                      2.0        3.0    3.0
D                      2.0        2.0    1.0
E                      1.0        2.0    1.0
general                1.0        2.0    2.0


## Precio Mercado

In [75]:
depa_df.loc[:, 'segmento'] = depa_df['categoria'].str[0].astype(str)
orden_segmentos = ['E', 'D', 'C', 'B', ]#'A', 'S']#, 'L'
grupo_descripcion_casa = depa_df.groupby('segmento')['precio'].describe().reset_index()
grupo_descripcion_casa = grupo_descripcion_casa.loc[:, ['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion_casa.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
# Ordenar el dataframe según el orden específico de segmentos
ordenado = grupo_descripcion_casa.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,346040.0,980000.0,6.355342e+05
1,D,1249000.0,1712133.0,1.406985e+06
2,C,2250000.0,2500000.0,2.295001e+06
3,B,2650000.0,2850000.0,2.783333e+06


In [76]:
# Función para formatear valores en formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
grupo_descripcion_casa 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\3321038986.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    $2,650,000.00
1    $2,250,000.00
2    $1,249,000.00
3      $346,040.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\3321038986.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    $2,850,000.00
1    $2,500,000.00
2    $1,712,133.00
3      $980,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppDa

,segmento,Mínimo,Máximo,Promedio
0,B,"$2,650,000.00","$2,850,000.00","$2,783,333.33"
1,C,"$2,250,000.00","$2,500,000.00","$2,295,001.20"
2,D,"$1,249,000.00","$1,712,133.00","$1,406,985.20"
3,E,"$346,040.00","$980,000.00","$635,534.23"


In [ ]:
# Paso 1: Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()

# Paso 2: Seleccionar las columnas deseadas
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']]

# Paso 3: Renombrar las columnas
#grupo_descripcion.rename(columns={
#    'min': 'Mínimo',
#    'max': 'Máximo',
#    'mean': 'Promedio',
#}, inplace=True)

# Paso 4: Calcular el percentil 95% para cada segmento
#percentiles_95 = casa_df.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Mostrar el resultado final
#print(grupo_descripcion)

In [81]:
# Filtrar los datos según las condiciones especificadas
#filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Calcular las estadísticas descriptivas
todo_descripcion = depa_df['precio'].describe()[['min', 'max', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general' # Agregar la columna de segmento
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Calcular el percentil 95%
#percentil_95 = filtrado_df['precio_m2_construido'].quantile(0.95)
# Reemplazar el valor máximo con el percentil 95%
#todo_descripcion['Máximo'] = percentil_95

# Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
print(todo_descripcion)

             Mínimo         Máximo       Promedio segmento
precio  $346,040.00  $2,850,000.00  $1,205,026.39  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\3749309691.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio    $346,040.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\3749309691.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio    $2,850,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\3749309691.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will rais

In [ ]:
# Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']] # Seleccionar las columnas deseadas
#grupo_descripcion.rename(columns={'min': 'Mínimo','max': 'Máximo', 'mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% para cada segmento
#percentiles_95 = df_total.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Paso 7: Calcular las estadísticas generales
#general_stats = df_total['precio'].describe()[['min', 'max', 'mean']].to_frame().T
#general_stats.rename(columns={'min': 'Mínimo','max': 'Máximo','mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% general
#general_percentil_95 = df_total['precio'].quantile(0.95)

# Reemplazar el máximo general con el percentil 95%
#general_stats['Máximo'] = general_percentil_95

# Agregar una fila para los resultados generales
#general_stats['segmento'] = 'General'

# Concatenar los resultados generales al DataFrame de segmentos
#resultado_final = pd.concat([grupo_descripcion, general_stats], ignore_index=True)
#print(resultado_final)

In [82]:
resultado_final = pd.concat([grupo_descripcion_casa, todo_descripcion], ignore_index=True)
print(resultado_final)

  segmento         Mínimo         Máximo       Promedio
0        B  $2,650,000.00  $2,850,000.00  $2,783,333.33
1        C  $2,250,000.00  $2,500,000.00  $2,295,001.20
2        D  $1,249,000.00  $1,712,133.00  $1,406,985.20
3        E    $346,040.00    $980,000.00    $635,534.23
4  general    $346,040.00  $2,850,000.00  $1,205,026.39


In [83]:
orden_segmentos = ['E', 'D', 'C', 'B','general']#, 'L' 'A', 'S',
ordenado = resultado_final.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,"$346,040.00","$980,000.00","$635,534.23"
1,D,"$1,249,000.00","$1,712,133.00","$1,406,985.20"
2,C,"$2,250,000.00","$2,500,000.00","$2,295,001.20"
3,B,"$2,650,000.00","$2,850,000.00","$2,783,333.33"
4,general,"$346,040.00","$2,850,000.00","$1,205,026.39"


In [84]:
# Guardar el JSON en un archivo
resultados_finales_desc_json = resultado_final.to_json(orient='index')
with open('assets/tablas/grupo_precio_mercado_depa.json', 'w') as f:
    f.write(resultados_finales_desc_json)

## M2 Terreno

In [86]:
depa_df.loc[:, 'segmento'] = depa_df['categoria'].str[0].astype(str)
orden_segmentos = ['E', 'D', 'C', 'B', ]#, 'L''A', 'S'
grupo_descripcion_casa = depa_df.groupby('segmento')['metros_total'].describe().reset_index()
grupo_descripcion_casa = grupo_descripcion_casa.loc[:, ['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion_casa.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
# Ordenar el dataframe según el orden específico de segmentos
ordenado = grupo_descripcion_casa.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,34.0,96.0,65.612903
1,D,80.0,120.0,93.400000
2,C,120.0,156.0,148.800000
3,B,95.0,110.0,100.000000


In [87]:
# Función para formatear valores en formato monetario por m2
def formato_monetario_m2(valor):
    return f"{valor:,.2f} m²"  # Formato con $, 2 decimales, separador de miles y unidad m²

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
grupo_descripcion_casa 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\3382106320.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     95.00 m²
1    120.00 m²
2     80.00 m²
3     34.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\3382106320.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    110.00 m²
1    156.00 m²
2    120.00 m²
3     96.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19

,segmento,Mínimo,Máximo,Promedio
0,B,95.00 m²,110.00 m²,100.00 m²
1,C,120.00 m²,156.00 m²,148.80 m²
2,D,80.00 m²,120.00 m²,93.40 m²
3,E,34.00 m²,96.00 m²,65.61 m²


In [ ]:
# Paso 1: Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()

# Paso 2: Seleccionar las columnas deseadas
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']]

# Paso 3: Renombrar las columnas
#grupo_descripcion.rename(columns={
#    'min': 'Mínimo',
#    'max': 'Máximo',
#    'mean': 'Promedio',
#}, inplace=True)

# Paso 4: Calcular el percentil 95% para cada segmento
#percentiles_95 = casa_df.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Mostrar el resultado final
#print(grupo_descripcion)

In [88]:
# Filtrar los datos según las condiciones especificadas
#filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Calcular las estadísticas descriptivas
todo_descripcion = depa_df['metros_total'].describe()[['min', 'max', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general' # Agregar la columna de segmento
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Calcular el percentil 95%
#percentil_95 = filtrado_df['precio_m2_construido'].quantile(0.95)
# Reemplazar el valor máximo con el percentil 95%
#todo_descripcion['Máximo'] = percentil_95


# Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"{valor:,.2f} m²"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
print(todo_descripcion)

                Mínimo     Máximo  Promedio segmento
metros_total  34.00 m²  156.00 m²  88.07 m²  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\2326618923.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    34.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\2326618923.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    156.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\2326618923.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will

In [ ]:
# Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']] # Seleccionar las columnas deseadas
#grupo_descripcion.rename(columns={'min': 'Mínimo','max': 'Máximo', 'mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% para cada segmento
#percentiles_95 = df_total.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Paso 7: Calcular las estadísticas generales
#general_stats = df_total['precio'].describe()[['min', 'max', 'mean']].to_frame().T
#general_stats.rename(columns={'min': 'Mínimo','max': 'Máximo','mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% general
#general_percentil_95 = df_total['precio'].quantile(0.95)

# Reemplazar el máximo general con el percentil 95%
#general_stats['Máximo'] = general_percentil_95

# Agregar una fila para los resultados generales
#general_stats['segmento'] = 'General'

# Concatenar los resultados generales al DataFrame de segmentos
#resultado_final = pd.concat([grupo_descripcion, general_stats], ignore_index=True)
#print(resultado_final)

In [89]:
resultado_final = pd.concat([grupo_descripcion_casa, todo_descripcion], ignore_index=True)
print(resultado_final)

  segmento     Mínimo     Máximo   Promedio
0        B   95.00 m²  110.00 m²  100.00 m²
1        C  120.00 m²  156.00 m²  148.80 m²
2        D   80.00 m²  120.00 m²   93.40 m²
3        E   34.00 m²   96.00 m²   65.61 m²
4  general   34.00 m²  156.00 m²   88.07 m²


In [90]:
orden_segmentos = ['E', 'D', 'C', 'B', 'general']#, 'L''A', 'S',
ordenado = resultado_final.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,34.00 m²,96.00 m²,65.61 m²
1,D,80.00 m²,120.00 m²,93.40 m²
2,C,120.00 m²,156.00 m²,148.80 m²
3,B,95.00 m²,110.00 m²,100.00 m²
4,general,34.00 m²,156.00 m²,88.07 m²


In [91]:
# Guardar el JSON en un archivo
resultados_finales_desc_json = resultado_final.to_json(orient='index')
with open('assets/tablas/grupo_terreno_depa.json', 'w') as f:
    f.write(resultados_finales_desc_json)

## M2 Construcción

In [93]:
depa_df.loc[:, 'segmento'] = depa_df['categoria'].str[0].astype(str)
orden_segmentos = ['E', 'D', 'C', 'B',]#, 'L' 'A', 'S'
grupo_descripcion_casa = depa_df.groupby('segmento')['metros_construido'].describe().reset_index()
grupo_descripcion_casa = grupo_descripcion_casa.loc[:, ['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion_casa.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
# Ordenar el dataframe según el orden específico de segmentos
ordenado = grupo_descripcion_casa.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,34.0,90.0,64.935484
1,D,80.0,120.0,93.400000
2,C,120.0,156.0,148.800000
3,B,95.0,110.0,100.000000


In [94]:
# Función para formatear valores en formato monetario por m2
def formato_monetario_m2(valor):
    return f"{valor:,.2f} m²"  # Formato con $, 2 decimales, separador de miles y unidad m²

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
grupo_descripcion_casa 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\3382106320.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     95.00 m²
1    120.00 m²
2     80.00 m²
3     34.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\3382106320.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    110.00 m²
1    156.00 m²
2    120.00 m²
3     90.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19

,segmento,Mínimo,Máximo,Promedio
0,B,95.00 m²,110.00 m²,100.00 m²
1,C,120.00 m²,156.00 m²,148.80 m²
2,D,80.00 m²,120.00 m²,93.40 m²
3,E,34.00 m²,90.00 m²,64.94 m²


In [ ]:
# Paso 1: Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()

# Paso 2: Seleccionar las columnas deseadas
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']]

# Paso 3: Renombrar las columnas
#grupo_descripcion.rename(columns={
#    'min': 'Mínimo',
#    'max': 'Máximo',
#    'mean': 'Promedio',
#}, inplace=True)

# Paso 4: Calcular el percentil 95% para cada segmento
#percentiles_95 = casa_df.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Mostrar el resultado final
#print(grupo_descripcion)

In [95]:
# Filtrar los datos según las condiciones especificadas
#filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Calcular las estadísticas descriptivas
todo_descripcion = depa_df['metros_construido'].describe()[['min', 'max', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general' # Agregar la columna de segmento
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Calcular el percentil 95%
#percentil_95 = filtrado_df['precio_m2_construido'].quantile(0.95)
# Reemplazar el valor máximo con el percentil 95%
#todo_descripcion['Máximo'] = percentil_95


# Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"{valor:,.2f} m²"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
print(todo_descripcion)

                     Mínimo     Máximo  Promedio segmento
metros_construido  34.00 m²  156.00 m²  87.69 m²  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\4103975992.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_construido    34.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\4103975992.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_construido    156.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\4103975992.py:24: FutureWarning: Setting an item of incompatible dtype is deprecate

In [ ]:
# Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']] # Seleccionar las columnas deseadas
#grupo_descripcion.rename(columns={'min': 'Mínimo','max': 'Máximo', 'mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% para cada segmento
#percentiles_95 = df_total.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Paso 7: Calcular las estadísticas generales
#general_stats = df_total['precio'].describe()[['min', 'max', 'mean']].to_frame().T
#general_stats.rename(columns={'min': 'Mínimo','max': 'Máximo','mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% general
#general_percentil_95 = df_total['precio'].quantile(0.95)

# Reemplazar el máximo general con el percentil 95%
#general_stats['Máximo'] = general_percentil_95

# Agregar una fila para los resultados generales
#general_stats['segmento'] = 'General'

# Concatenar los resultados generales al DataFrame de segmentos
#resultado_final = pd.concat([grupo_descripcion, general_stats], ignore_index=True)
#print(resultado_final)

In [96]:
resultado_final = pd.concat([grupo_descripcion_casa, todo_descripcion], ignore_index=True)
print(resultado_final)

  segmento     Mínimo     Máximo   Promedio
0        B   95.00 m²  110.00 m²  100.00 m²
1        C  120.00 m²  156.00 m²  148.80 m²
2        D   80.00 m²  120.00 m²   93.40 m²
3        E   34.00 m²   90.00 m²   64.94 m²
4  general   34.00 m²  156.00 m²   87.69 m²


In [97]:
orden_segmentos = ['E', 'D', 'C', 'B', 'general']#, 'L''A', 'S',
ordenado = resultado_final.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,34.00 m²,90.00 m²,64.94 m²
1,D,80.00 m²,120.00 m²,93.40 m²
2,C,120.00 m²,156.00 m²,148.80 m²
3,B,95.00 m²,110.00 m²,100.00 m²
4,general,34.00 m²,156.00 m²,87.69 m²


In [98]:
# Guardar el JSON en un archivo
resultados_finales_desc_json = resultado_final.to_json(orient='index')
with open('assets/tablas/grupo_construidos_depa.json', 'w') as f:
    f.write(resultados_finales_desc_json)

## Pecio M2 Construcción

In [99]:
depa_df['precio_m2_construido']

16     30000.00
22     20416.67
24     14423.08
33     20879.67
37     19047.41
42     10833.33
44     13888.89
46     13147.37
62     11951.22
64     10666.67
65     11294.12
77     13373.17
82      9146.34
90      7730.31
95     10333.33
100    10909.09
121    11304.35
127     8750.00
128    10243.90
131     3977.47
231    14673.91
234    11166.67
258     3977.47
259     9716.98
260    11304.35
263    10333.33
264    10333.33
265     7730.31
266     9146.34
268    11294.12
269    11951.22
270    13147.37
271    13888.89
272    10833.33
273    19047.41
274    20833.33
275    30000.00
277    11764.71
278    10227.27
279    10194.90
280     9583.33
290    20879.67
326    24090.91
354    14423.08
355    14423.08
356    14423.08
357    14423.08
358    14423.08
359    14423.10
360    14423.13
469    11166.67
588    10333.33
707    10333.33
806     6499.56
Name: precio_m2_construido, dtype: float64

In [101]:
depa_df.loc[:, 'segmento'] = depa_df['categoria'].str[0].astype(str)
orden_segmentos = ['E', 'D', 'C', 'B',]#, 'L' 'A', 'S'
grupo_descripcion_casa = depa_df.groupby('segmento')['precio_m2_construido'].describe().reset_index()
grupo_descripcion_casa = grupo_descripcion_casa.loc[:, ['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion_casa.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
# Ordenar el dataframe según el orden específico de segmentos
ordenado = grupo_descripcion_casa.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,3977.47,14673.91,10045.519355
1,D,10833.33,20879.67,15559.334000
2,C,14423.08,20833.33,15663.471000
3,B,24090.91,30000.00,28030.303333


In [102]:
# Función para formatear valores en formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
grupo_descripcion_casa 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\3321038986.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    $24,090.91
1    $14,423.08
2    $10,833.33
3     $3,977.47
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\3321038986.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    $30,000.00
1    $20,833.33
2    $20,879.67
3    $14,673.91
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_

,segmento,Mínimo,Máximo,Promedio
0,B,"$24,090.91","$30,000.00","$28,030.30"
1,C,"$14,423.08","$20,833.33","$15,663.47"
2,D,"$10,833.33","$20,879.67","$15,559.33"
3,E,"$3,977.47","$14,673.91","$10,045.52"


In [ ]:
# Paso 1: Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()

# Paso 2: Seleccionar las columnas deseadas
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']]

# Paso 3: Renombrar las columnas
#grupo_descripcion.rename(columns={
#    'min': 'Mínimo',
#    'max': 'Máximo',
#    'mean': 'Promedio',
#}, inplace=True)

# Paso 4: Calcular el percentil 95% para cada segmento
#percentiles_95 = casa_df.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Mostrar el resultado final
#print(grupo_descripcion)

In [103]:
# Filtrar los datos según las condiciones especificadas
#filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Calcular las estadísticas descriptivas
todo_descripcion = depa_df['precio_m2_construido'].describe()[['min', 'max', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general' # Agregar la columna de segmento
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Calcular el percentil 95%
#percentil_95 = filtrado_df['precio_m2_construido'].quantile(0.95)
# Reemplazar el valor máximo con el percentil 95%
#todo_descripcion['Máximo'] = percentil_95

# Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
print(todo_descripcion)

                         Mínimo      Máximo    Promedio segmento
precio_m2_construido  $3,977.47  $30,000.00  $13,106.11  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\1082635710.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $3,977.47
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\1082635710.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $30,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19968\1082635710.py:23: FutureWarning: Setting an item of incompatible dtype is d

In [ ]:
# Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']] # Seleccionar las columnas deseadas
#grupo_descripcion.rename(columns={'min': 'Mínimo','max': 'Máximo', 'mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% para cada segmento
#percentiles_95 = df_total.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Paso 7: Calcular las estadísticas generales
#general_stats = df_total['precio'].describe()[['min', 'max', 'mean']].to_frame().T
#general_stats.rename(columns={'min': 'Mínimo','max': 'Máximo','mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% general
#general_percentil_95 = df_total['precio'].quantile(0.95)

# Reemplazar el máximo general con el percentil 95%
#general_stats['Máximo'] = general_percentil_95

# Agregar una fila para los resultados generales
#general_stats['segmento'] = 'General'

# Concatenar los resultados generales al DataFrame de segmentos
#resultado_final = pd.concat([grupo_descripcion, general_stats], ignore_index=True)
#print(resultado_final)

In [104]:
resultado_final = pd.concat([grupo_descripcion_casa, todo_descripcion], ignore_index=True)
print(resultado_final)

  segmento      Mínimo      Máximo    Promedio
0        B  $24,090.91  $30,000.00  $28,030.30
1        C  $14,423.08  $20,833.33  $15,663.47
2        D  $10,833.33  $20,879.67  $15,559.33
3        E   $3,977.47  $14,673.91  $10,045.52
4  general   $3,977.47  $30,000.00  $13,106.11


In [105]:
orden_segmentos = ['E', 'D', 'C', 'B','general']#, 'L', 'A', 'S'
ordenado = resultado_final.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,"$3,977.47","$14,673.91","$10,045.52"
1,D,"$10,833.33","$20,879.67","$15,559.33"
2,C,"$14,423.08","$20,833.33","$15,663.47"
3,B,"$24,090.91","$30,000.00","$28,030.30"
4,general,"$3,977.47","$30,000.00","$13,106.11"


In [106]:
# Guardar el JSON en un archivo
resultados_finales_desc_json = resultado_final.to_json(orient='index')
with open('assets/tablas/grupo_precio_construidos_depa.json', 'w') as f:
    f.write(resultados_finales_desc_json)

# 
_____________
# ANTES

In [13]:
df_total['segmento'] = df_total['categoria'].str[0]
grupo_descripcion = df_total.groupby('segmento')['precio'].describe().reset_index()
grupo_descripcion = grupo_descripcion[['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
grupo_descripcion 

,segmento,Mínimo,Máximo,Promedio
0,A,3300000.0,3946465.0,3.582541e+06
1,B,2600000.0,3200000.0,2.883613e+06
2,C,1800000.0,2499000.0,2.131160e+06
3,D,1020000.0,1737200.0,1.278118e+06
4,E,346040.0,995000.0,6.531739e+05
5,L,17000000.0,17500000.0,1.725000e+07
6,S,500000.0,10000000.0,3.034541e+06


In [14]:

# Paso 1: Agrupar por segmento y calcular las estadísticas descriptivas
grupo_descripcion = df_total.groupby('segmento')['precio'].describe().reset_index()

# Paso 2: Seleccionar las columnas deseadas
grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']]

# Paso 3: Renombrar las columnas
grupo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Paso 4: Calcular el percentil 95% para cada segmento
percentiles_95 = df_total.groupby('segmento')['precio'].quantile(0.95).reset_index()
percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Mostrar el resultado final
print(grupo_descripcion)


  segmento      Mínimo      Máximo      Promedio
0        A   3300000.0   3946465.0  3.582541e+06
1        B   2600000.0   3200000.0  2.883613e+06
2        C   1800000.0   2480000.0  2.131160e+06
3        D   1020000.0   1700000.0  1.278118e+06
4        E    346040.0    980000.0  6.531739e+05
5        L  17000000.0  17500000.0  1.725000e+07
6        S    500000.0   7050000.0  3.034541e+06


In [15]:
# Función para formatear valores en formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion.iloc[:, i] = grupo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
grupo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18412\1329763181.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $3,300,000.00
1     $2,600,000.00
2     $1,800,000.00
3     $1,020,000.00
4       $346,040.00
5    $17,000,000.00
6       $500,000.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion.iloc[:, i] = grupo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18412\1329763181.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $3,946,465.00
1     $3,200,000.00
2     $2,480,000.00
3     $1,700,000.00
4       $980,000.00
5    $17,500,000.00
6     $7,050,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descr

,segmento,Mínimo,Máximo,Promedio
0,A,"$3,300,000.00","$3,946,465.00","$3,582,540.95"
1,B,"$2,600,000.00","$3,200,000.00","$2,883,612.90"
2,C,"$1,800,000.00","$2,480,000.00","$2,131,159.98"
3,D,"$1,020,000.00","$1,700,000.00","$1,278,118.23"
4,E,"$346,040.00","$980,000.00","$653,173.93"
5,L,"$17,000,000.00","$17,500,000.00","$17,250,000.00"
6,S,"$500,000.00","$7,050,000.00","$3,034,540.54"


In [16]:
import pandas as pd

# Supongamos que df_total ya está definido y contiene las columnas 'metros_construido' y 'precio_m2_construido'.

# Filtrar los datos según las condiciones especificadas
filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Paso 1: Calcular las estadísticas descriptivas
todo_descripcion = filtrado_df['precio_m2_construido'].describe()[['min', 'max', 'mean']].to_frame().T

# Agregar la columna de segmento
todo_descripcion['segmento'] = 'general'

# Paso 2: Renombrar las columnas
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Paso 3: Calcular el percentil 95%
percentil_95 = filtrado_df['precio_m2_construido'].quantile(0.95)

# Paso 4: Reemplazar el valor máximo con el percentil 95%
todo_descripcion['Máximo'] = percentil_95

# Paso 5: Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)

# Mostrar el resultado final
print(todo_descripcion)


                     Mínimo Máximo Promedio segmento
precio_m2_construido   $nan   $nan     $nan  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_18412\79234579.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $nan
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18412\79234579.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $nan
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18412\79234579.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and wil

In [17]:
import pandas as pd

# Supongamos que df_total ya está definido y contiene las columnas 'segmento' y 'precio'.

# Paso 1: Agrupar por segmento y calcular las estadísticas descriptivas
grupo_descripcion = df_total.groupby('segmento')['precio'].describe().reset_index()

# Paso 2: Seleccionar las columnas deseadas
grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']]

# Paso 3: Renombrar las columnas
grupo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Paso 4: Calcular el percentil 95% para cada segmento
percentiles_95 = df_total.groupby('segmento')['precio'].quantile(0.95).reset_index()
percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Paso 7: Calcular las estadísticas generales
general_stats = df_total['precio'].describe()[['min', 'max', 'mean']].to_frame().T
general_stats.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Calcular el percentil 95% general
general_percentil_95 = df_total['precio'].quantile(0.95)

# Reemplazar el máximo general con el percentil 95%
general_stats['Máximo'] = general_percentil_95

# Agregar una fila para los resultados generales
general_stats['segmento'] = 'General'

# Concatenar los resultados generales al DataFrame de segmentos
resultado_final = pd.concat([grupo_descripcion, general_stats], ignore_index=True)

# Mostrar el resultado final
print(resultado_final)


  segmento      Mínimo      Máximo      Promedio
0        A   3300000.0   3946465.0  3.582541e+06
1        B   2600000.0   3200000.0  2.883613e+06
2        C   1800000.0   2480000.0  2.131160e+06
3        D   1020000.0   1700000.0  1.278118e+06
4        E    346040.0    980000.0  6.531739e+05
5        L  17000000.0  17500000.0  1.725000e+07
6        S    500000.0   7050000.0  3.034541e+06
7  General    346040.0   3946465.0  1.474366e+06


In [18]:
# Función para formatear valores en formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    resultado_final.iloc[:, i] = resultado_final.iloc[:, i].astype(float).apply(formato_monetario)
resultado_final 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18412\2150460987.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $3,300,000.00
1     $2,600,000.00
2     $1,800,000.00
3     $1,020,000.00
4       $346,040.00
5    $17,000,000.00
6       $500,000.00
7       $346,040.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  resultado_final.iloc[:, i] = resultado_final.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18412\2150460987.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $3,946,465.00
1     $3,200,000.00
2     $2,480,000.00
3     $1,700,000.00
4       $980,000.00
5    $17,500,000.00
6     $7,050,000.00
7     $3,946,465.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a c

,segmento,Mínimo,Máximo,Promedio
0,A,"$3,300,000.00","$3,946,465.00","$3,582,540.95"
1,B,"$2,600,000.00","$3,200,000.00","$2,883,612.90"
2,C,"$1,800,000.00","$2,480,000.00","$2,131,159.98"
3,D,"$1,020,000.00","$1,700,000.00","$1,278,118.23"
4,E,"$346,040.00","$980,000.00","$653,173.93"
5,L,"$17,000,000.00","$17,500,000.00","$17,250,000.00"
6,S,"$500,000.00","$7,050,000.00","$3,034,540.54"
7,General,"$346,040.00","$3,946,465.00","$1,474,365.53"


In [19]:
# Guardar el JSON en un archivo
#resultados_finales_descripcion = pd.concat([grupo_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_descripcion = resultado_final
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_precio_mercado.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [20]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio
0,A,"$3,300,000.00","$3,946,465.00","$3,582,540.95"
1,B,"$2,600,000.00","$3,200,000.00","$2,883,612.90"
2,C,"$1,800,000.00","$2,480,000.00","$2,131,159.98"
3,D,"$1,020,000.00","$1,700,000.00","$1,278,118.23"
4,E,"$346,040.00","$980,000.00","$653,173.93"
5,L,"$17,000,000.00","$17,500,000.00","$17,250,000.00"
6,S,"$500,000.00","$7,050,000.00","$3,034,540.54"
7,General,"$346,040.00","$3,946,465.00","$1,474,365.53"


In [15]:
orden_segmentos = ["E", "D", "C", "B", "A", "S", "L", "ELITE", "general"]

# Función para limpiar valores con unidades y convertirlos a float
def limpiar_valor(valor):
    # Eliminar unidades (m², $, etc.) y convertir a float
    if isinstance(valor, str):
        return float(valor.replace(" m²", "").replace("$", "").replace(",", ""))
    else:
        return valor

# Aplicar limpieza de valores en las columnas que contienen medidas
for columna in ["Mínimo", "Máximo", "Promedio", "Mediana"]:
    resultados_finales_descripcion[columna] = resultados_finales_descripcion[columna].apply(limpiar_valor)
# Función para formatear nuevamente los valores en el formato deseado
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Volver a aplicar el formato monetario después de los ajustes
for columna in ["Mínimo", "Máximo", "Promedio", "Mediana"]:
    resultados_finales_descripcion[columna] = resultados_finales_descripcion[columna].apply(formato_monetario)



In [16]:
# Limpiar valores y convertir a float
def limpiar_valor(valor):
    return float(valor.replace(" m²", "").replace("$", "").replace(",", ""))

for columna in ["Mínimo", "Máximo", "Promedio", "Mediana"]:
    resultados_finales_descripcion[columna] = resultados_finales_descripcion[columna].apply(limpiar_valor)

# Verificar el orden de los segmentos
orden_segmentos = ["E", "D", "C", "B", "A", "S", "L", "ELITE", "general"]
resultados_finales_descripcion = resultados_finales_descripcion.set_index('segmento').reindex(orden_segmentos).reset_index()

# Validar que los valores no sean inconsistentes
for i in range(1, len(resultados_finales_descripcion)):
    for columna in ["Mínimo", "Máximo", "Promedio", "Mediana"]:
        valor_actual = resultados_finales_descripcion.loc[i, columna]
        valor_anterior = resultados_finales_descripcion.loc[i - 1, columna]
        
        # Ajustar valores inconsistentes
        if valor_actual < valor_anterior:
            resultados_finales_descripcion.loc[i, columna] = valor_anterior

# Volver a formatear los valores a formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"

for columna in ["Mínimo", "Máximo", "Promedio", "Mediana"]:
    resultados_finales_descripcion[columna] = resultados_finales_descripcion[columna].apply(formato_monetario)

# Guardar los resultados en un archivo JSON
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='records', force_ascii=False)
with open('assets/tablas/grupo_precio_mercado_ajustado.json', 'w', encoding='utf-8') as f:
    f.write(resultados_finales_desc_json)


In [21]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio
0,A,"$3,300,000.00","$3,946,465.00","$3,582,540.95"
1,B,"$2,600,000.00","$3,200,000.00","$2,883,612.90"
2,C,"$1,800,000.00","$2,480,000.00","$2,131,159.98"
3,D,"$1,020,000.00","$1,700,000.00","$1,278,118.23"
4,E,"$346,040.00","$980,000.00","$653,173.93"
5,L,"$17,000,000.00","$17,500,000.00","$17,250,000.00"
6,S,"$500,000.00","$7,050,000.00","$3,034,540.54"
7,General,"$346,040.00","$3,946,465.00","$1,474,365.53"


# M2 Terreno

In [22]:
df_total = df_total[df_total['metros_construido'] > 40]
df_total_terreno = df_total[~df_total['metros_total'].isin([0, 1, 2])]

df_total_terreno = df_total_terreno[(df_total_terreno['metros_total'] != 1) & (df_total_terreno['metros_total'] > 20)]

df_descripcion = df_total_terreno.groupby('segmento')['metros_total'].describe(percentiles=[0.96]).reset_index()
df_descripcion = df_descripcion[['segmento', 'min', '96%', 'mean']] #Columnas deseadas
df_descripcion.rename(columns={
    'min': 'Mínimo',
    '96%': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)


# Función para formatear valores en formato metros
def formato_metros(valor):
    return f"{valor:,.2f} m²"

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
df_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18412\1393368157.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0      150.00 m²
1       70.00 m²
2       80.00 m²
3       70.00 m²
4       41.00 m²
5    1,000.00 m²
6       82.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18412\1393368157.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    1,049.00 m²
1      232.00 m²
2      400.00 m²
3      398.00 m²
4      266.00 m²
5    1,972.00 m²
6    1,692.80 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype

,segmento,Mínimo,Máximo,Promedio
0,A,150.00 m²,"1,049.00 m²",297.19 m²
1,B,70.00 m²,232.00 m²,164.65 m²
2,C,80.00 m²,400.00 m²,172.00 m²
3,D,70.00 m²,398.00 m²,137.77 m²
4,E,41.00 m²,266.00 m²,138.55 m²
5,L,"1,000.00 m²","1,972.00 m²","1,486.00 m²"
6,S,82.00 m²,"1,692.80 m²",305.96 m²


In [24]:
todo_descripcion = df_total[~df_total['metros_total'].isin([0, 1, 2])]['metros_total'].describe(percentiles=[0.95])[['min', '95%', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    'min': 'Mínimo',
    '95%': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(3):  # 0, 1, 2 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18412\1067183333.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    41.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18412\1067183333.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    350.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18412\1067183333.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise

,Mínimo,Máximo,Promedio,segmento
metros_total,41.00 m²,350.00 m²,175.15 m²,general


In [25]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([df_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_terreno.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [26]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio
0,A,150.00 m²,"1,049.00 m²",297.19 m²
1,B,70.00 m²,232.00 m²,164.65 m²
2,C,80.00 m²,400.00 m²,172.00 m²
3,D,70.00 m²,398.00 m²,137.77 m²
4,E,41.00 m²,266.00 m²,138.55 m²
5,L,"1,000.00 m²","1,972.00 m²","1,486.00 m²"
6,S,82.00 m²,"1,692.80 m²",305.96 m²
7,general,41.00 m²,350.00 m²,175.15 m²


# M2 Construcción

In [27]:
df_total_terreno = df_total[(df_total['metros_construido'] > 40) & (~df_total['metros_construido'].isin([0, 1,2,3]))]
df_descripcion = df_total_terreno.groupby('segmento')['metros_construido'].describe(percentiles=[0.95]).reset_index()
df_descripcion = df_descripcion[['segmento', 'min', '95%', 'mean']] #Columnas deseadas
df_descripcion.rename(columns={
    '95%': 'Máximo',
    'min': 'Mínimo',
    'mean': 'Promedio',
}, inplace=True)


# Función para formatear valores en formato metros
def formato_metros(valor):
    return f"{valor:,.2f} m²"

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
df_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18412\1810280376.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    150.00 m²
1     95.00 m²
2     90.00 m²
3     62.00 m²
4     41.00 m²
5    500.00 m²
6     81.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18412\1810280376.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    360.00 m²
1    600.00 m²
2    404.00 m²
3    398.00 m²
4    371.00 m²
5    520.00 m²
6    425.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros

,segmento,Mínimo,Máximo,Promedio
0,A,150.00 m²,360.00 m²,253.67 m²
1,B,95.00 m²,600.00 m²,204.18 m²
2,C,90.00 m²,404.00 m²,202.16 m²
3,D,62.00 m²,398.00 m²,139.65 m²
4,E,41.00 m²,371.00 m²,172.47 m²
5,L,500.00 m²,520.00 m²,510.00 m²
6,S,81.00 m²,425.00 m²,247.52 m²


In [28]:
todo_descripcion = df_total[~df_total['metros_construido'].isin([0,1,2,3]) & (df_total['metros_total'] > 3)]['metros_total'].describe(percentiles=[0.95])[['min', '95%', 'mean',]].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    '95%': 'Máximo',
    'min': 'Mínimo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Máximo Total', 'Promedio'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18412\2220166576.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    41.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18412\2220166576.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    350.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18412\2220166576.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise

,Mínimo,Máximo,Promedio,segmento
metros_total,41.00 m²,350.00 m²,175.15 m²,general


In [29]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([df_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_construidos.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [30]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio
0,A,150.00 m²,360.00 m²,253.67 m²
1,B,95.00 m²,600.00 m²,204.18 m²
2,C,90.00 m²,404.00 m²,202.16 m²
3,D,62.00 m²,398.00 m²,139.65 m²
4,E,41.00 m²,371.00 m²,172.47 m²
5,L,500.00 m²,520.00 m²,510.00 m²
6,S,81.00 m²,425.00 m²,247.52 m²
7,general,41.00 m²,350.00 m²,175.15 m²


# Pecio M2 Construcción

In [31]:
df_total_terreno = df_total[~df_total['metros_construido'].isin([0, 1,2,3])]
df_total_terreno = df_total_terreno[df_total_terreno['precio_m2_construido'] > 10]
grupo_filtrado = df_total_terreno.groupby('segmento')['precio_m2_construido'].describe(percentiles=[0.95]).reset_index()
# Mantener solo las columnas que deseas (en este caso, min, 95%, mean)
grupo_filtrado = grupo_filtrado[['segmento', 'min', '95%', 'mean']] #Columnas deseadas
grupo_filtrado.rename(columns={
    '95%': 'Máximo',
    'min': 'Mínimo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float).apply(formato_monetario)

grupo_filtrado 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18412\952371004.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $8,912.50
1     $5,333.33
2     $4,918.81
3     $2,607.14
4     $1,023.81
5    $33,653.85
6     $1,562.50
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18412\952371004.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    $26,309.77
1    $22,857.14
2    $16,467.63
3    $18,548.39
4    $13,782.80
5    $34,000.00
6    $17,916.67
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float).apply

,segmento,Mínimo,Máximo,Promedio
0,A,"$8,912.50","$26,309.77","$15,535.85"
1,B,"$5,333.33","$22,857.14","$17,379.56"
2,C,"$4,918.81","$16,467.63","$12,082.48"
3,D,"$2,607.14","$18,548.39","$11,328.20"
4,E,"$1,023.81","$13,782.80","$5,536.80"
5,L,"$33,653.85","$34,000.00","$33,826.93"
6,S,"$1,562.50","$17,916.67","$12,057.03"


In [32]:
todo_descripcion = df_total[(df_total['precio_m2_construido'] > 6) & (~df_total['metros_construido'].isin([0, 1,2,3]))]['precio_m2_construido'].describe(percentiles=[0.95])[['min', '95%', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general'
todo_descripcion = todo_descripcion[['segmento', 'min', '95%', 'mean']] #Columnas deseadas

todo_descripcion.rename(columns={
    '95%': 'Máximo',
    'min': 'Mínimo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18412\4241514475.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $1,023.81
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18412\4241514475.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $19,333.33
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18412\4241514475.py:13: FutureWarning: Setting an item of incompatible dtype is d

,segmento,Mínimo,Máximo,Promedio
precio_m2_construido,general,"$1,023.81","$19,333.33","$8,945.61"


In [33]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([grupo_filtrado, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_precio_construidos.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [34]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio
0,A,"$8,912.50","$26,309.77","$15,535.85"
1,B,"$5,333.33","$22,857.14","$17,379.56"
2,C,"$4,918.81","$16,467.63","$12,082.48"
3,D,"$2,607.14","$18,548.39","$11,328.20"
4,E,"$1,023.81","$13,782.80","$5,536.80"
5,L,"$33,653.85","$34,000.00","$33,826.93"
6,S,"$1,562.50","$17,916.67","$12,057.03"
7,general,"$1,023.81","$19,333.33","$8,945.61"
